In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

real_csv = "/home/nikki/egfr_lowdata_scoring/data/ligands_real/EGFR_activities.csv"

real_df = pd.read_csv(real_csv)
real_df = real_df[["molecule_chembl_id", "canonical_smiles"]].dropna().drop_duplicates()

real_ids = set(real_df["molecule_chembl_id"])

props = []
for smi in real_df["canonical_smiles"]:
    mol = Chem.MolFromSmiles(smi)
    if mol:
        props.append([
            Descriptors.MolWt(mol),
            Descriptors.MolLogP(mol),
            Descriptors.NumHDonors(mol),
            Descriptors.NumHAcceptors(mol)
        ])

props_df = pd.DataFrame(props, columns=["MW", "logP", "HBD", "HBA"])

# Define filters (mean ± tolerance)
mw_min, mw_max = props_df.MW.mean() - 50, props_df.MW.mean() + 50
logp_min, logp_max = props_df.logP.mean() - 1, props_df.logP.mean() + 1
hbd_min, hbd_max = props_df.HBD.mean() - 1, props_df.HBD.mean() + 1
hba_min, hba_max = props_df.HBA.mean() - 1, props_df.HBA.mean() + 1


In [ ]:
#fecth chembl molecules
import requests

def fetch_chembl_molecules(limit=5000):
    url = "https://www.ebi.ac.uk/chembl/api/data/molecule"
    molecules = []
    offset = 0
    page_size = 1000

    while len(molecules) < limit:
        params = {
            "limit": page_size,
            "offset": offset,
            "format": "json"
        }
        r = requests.get(url, params=params)
        r.raise_for_status()
        data = r.json()["molecules"]

        for mol in data:
            if mol.get("molecule_structures"):
                molecules.append({
                    "molecule_chembl_id": mol["molecule_chembl_id"],
                    "smiles": mol["molecule_structures"]["canonical_smiles"]
                })

        offset += page_size
        print(f"Fetched {len(molecules)} molecules")

    return pd.DataFrame(molecules[:limit])


In [5]:
chembl_df = fetch_chembl_molecules(limit=5000)


Fetched 969 molecules
Fetched 1968 molecules
Fetched 2968 molecules
Fetched 3967 molecules
Fetched 4963 molecules
Fetched 5963 molecules


In [15]:
chembl_df = chembl_df[
    ~chembl_df["molecule_chembl_id"].isin(real_ids)
].reset_index(drop=True)


In [16]:
rows = []

for _, row in chembl_df.iterrows():
    mol = Chem.MolFromSmiles(row["smiles"])
    if not mol:
        continue

    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)

    if (
        mw_min <= mw <= mw_max and
        logp_min <= logp <= logp_max and
        hbd_min <= hbd <= hbd_max and
        hba_min <= hba <= hba_max
    ):
        rows.append(row)

synthetic_df = pd.DataFrame(rows)


In [18]:
# Sample up to 1000 synthetic ligands
synthetic_df = synthetic_df.sample(
    n=min(1000, len(synthetic_df)),
    random_state=42
).reset_index(drop=True)

# Assign synthetic IDs
synthetic_df["synthetic_id"] = [
    f"SYN{i:06d}" for i in range(1, len(synthetic_df) + 1)
]

# Output path
out_smi = "/home/nikki/egfr_lowdata_scoring/data/ligands_synthetic/synthetic.smi"

# Write .smi file (SMILES + ID)
with open(out_smi, "w") as f:
    for _, row in synthetic_df.iterrows():
        f.write(f"{row['smiles']} {row['synthetic_id']}\n")

print(f"Saved {len(synthetic_df)} synthetic ligands to {out_smi}")


Saved 194 synthetic ligands to /home/nikki/egfr_lowdata_scoring/data/ligands_synthetic/synthetic.smi


In [19]:
from rdkit import Chem
from rdkit.Chem import AllChem
import os

input_smi = "/home/nikki/egfr_lowdata_scoring/data/ligands_synthetic/synthetic.smi"
out_sdf_dir = "/home/nikki/egfr_lowdata_scoring/data/ligands_synthetic/sdf"

os.makedirs(out_sdf_dir, exist_ok=True)

with open(input_smi) as f:
    for line in f:
        smi, lig_id = line.strip().split()

        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            print(f"Skipping invalid SMILES: {lig_id}")
            continue

        mol = Chem.AddHs(mol)

        # Embed 3D
        status = AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        if status != 0:
            print(f"Embedding failed: {lig_id}")
            continue

        # Minimize
        AllChem.UFFOptimizeMolecule(mol)

        # Write SDF
        out_sdf = os.path.join(out_sdf_dir, f"{lig_id}.sdf")
        writer = Chem.SDWriter(out_sdf)
        writer.write(mol)
        writer.close()


[16:16:11] UFFTYPER: Unrecognized charge state for atom: 8
[16:16:11] UFFTYPER: Unrecognized charge state for atom: 8
[16:16:14] UFFTYPER: Unrecognized charge state for atom: 7
[16:16:14] UFFTYPER: Unrecognized charge state for atom: 7


In [ ]:
import os
import pandas as pd
import re

LOG_DIR = "/home/nikki/egfr_lowdata_scoring/data/docking/results/synth/logs"
SMI_FILE = "/home/nikki/egfr_lowdata_scoring/data/ligands_synthetic/synthetic.smi"
OUT_CSV = "/home/nikki/egfr_lowdata_scoring/results/synth/features_synth_raw.csv"

# Load SMILES
smiles_dict = {}
with open(SMI_FILE) as f:
    for line in f:
        smi, sid = line.strip().split()
        smiles_dict[sid] = smi

rows = []

for log_file in os.listdir(LOG_DIR):
    if not log_file.endswith("_log.txt"):
        continue

    lig_id = log_file.replace("_log.txt", "")
    log_path = os.path.join(LOG_DIR, log_file)

    with open(log_path) as f:
        for line in f:
            if re.match(r"\s*1\s+-?\d+\.\d+", line):
                score = float(line.split()[1])
                rows.append({
                    "synthetic_id": lig_id,
                    "smiles": smiles_dict.get(lig_id, None),
                    "docking_score": score
                })
                break

df = pd.DataFrame(rows)
df.to_csv(OUT_CSV, index=False)

print(f"Saved {len(df)} rows to {OUT_CSV}")


Saved 186 rows to /home/nikki/egfr_lowdata_scoring/data/features_synth_raw.csv


In [ ]:
#compute_features_synth
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, rdMolDescriptors, AllChem
from rdkit.DataStructs import ConvertToNumpyArray
import numpy as np

# Paths
RAW_CSV = "/home/nikki/egfr_lowdata_scoring/results/synth/features_synth_raw.csv"
OUT_CSV = "/home/nikki/egfr_lowdata_scoring/results/synth/features_synth.csv"

# Load docking results
df = pd.read_csv(RAW_CSV)

rows = []

for _, row in df.iterrows():
    lig_id = row["synthetic_id"]
    smi = row["smiles"]
    docking_score = row["docking_score"]

    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        continue

    # ---- PhysChem descriptors (SAME as Week 1) ----
    MW = Descriptors.MolWt(mol)
    LogP = Crippen.MolLogP(mol)
    HBD = rdMolDescriptors.CalcNumHBD(mol)
    HBA = rdMolDescriptors.CalcNumHBA(mol)
    TPSA = rdMolDescriptors.CalcTPSA(mol)

    # ---- Morgan fingerprint ----
    fp = AllChem.GetMorganFingerprintAsBitVect(
        mol,
        radius=2,
        nBits=1024
    )
    fp_array = np.zeros((1024,), dtype=int)
    ConvertToNumpyArray(fp, fp_array)

    # ---- Combine everything ----
    feature_row = {
        "ligand_id": lig_id,
        "label": 0,  # synthetic = non-binder / unknown
        "docking_score": docking_score,
        "MW": MW,
        "LogP": LogP,
        "HBD": HBD,
        "HBA": HBA,
        "TPSA": TPSA
    }

    # Add fingerprint bits
    for i in range(1024):
        feature_row[f"fp_{i}"] = fp_array[i]

    rows.append(feature_row)

# Save
features_df = pd.DataFrame(rows)
features_df.to_csv(OUT_CSV, index=False)

print(f"Saved {len(features_df)} synthetic feature rows to {OUT_CSV}")


Saved 186 synthetic feature rows to /home/nikki/egfr_lowdata_scoring/data/features_synth.csv


In [9]:
#ML model training

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score


In [10]:
real_df = pd.read_csv("/home/nikki/egfr_lowdata_scoring/data/features_real.csv")
synth_df = pd.read_csv("/home/nikki/egfr_lowdata_scoring/data/features_synth.csv")

descriptor_cols = ["docking_score"]
fp_cols = [c for c in real_df.columns if c.startswith("FP_")]

feature_cols = descriptor_cols + fp_cols


In [11]:
X_real = real_df[feature_cols].copy()
y_real = real_df["label"].astype(int)

X_synth = synth_df[feature_cols].copy()

X_real_train, X_real_test, y_real_train, y_real_test = train_test_split(
    X_real,
    y_real,
    test_size=0.2,
    stratify=y_real,
    random_state=42
)

In [12]:
rf_real = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced"
)

rf_real.fit(X_real_train, y_real_train)

probs_real = rf_real.predict_proba(X_real_test)[:, 1]

auc_real = roc_auc_score(y_real_test, probs_real)
pr_real = average_precision_score(y_real_test, probs_real)
acc_real = accuracy_score(y_real_test, probs_real > 0.5)

print("Baseline 1 – Real only")
print("ROC-AUC:", auc_real)
print("PR-AUC:", pr_real)
print("Accuracy:", acc_real)


Baseline 1 – Real only
ROC-AUC: 0.8333333333333334
PR-AUC: 0.9028571428571428
Accuracy: 0.625


In [13]:
synth_df["pseudo_label"] = np.nan

synth_df.loc[synth_df["docking_score"] <= -9.0, "pseudo_label"] = 1
synth_df.loc[synth_df["docking_score"] >= -7.0, "pseudo_label"] = 0

# Drop ambiguous synthetic ligands
synth_labeled = synth_df.dropna(subset=["pseudo_label"])


In [16]:
X_synth_train = synth_labeled[feature_cols]
y_synth_train = synth_labeled["pseudo_label"].astype(int)

X_train_aug = pd.concat([X_real_train, X_synth_train], axis=0)
y_train_aug = pd.concat([y_real_train, y_synth_train], axis=0)



In [19]:
rf_aug = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced"
)

rf_aug.fit(X_train_aug, y_train_aug)

probs_aug = rf_aug.predict_proba(X_real_test)[:, 1]

auc_aug = roc_auc_score(y_real_test, probs_aug)
pr_aug = average_precision_score(y_real_test, probs_aug)
acc_aug = accuracy_score(y_real_test, probs_aug > 0.5)

print("\nBaseline 2 - Real + Synthetic (pseudo-labels)")
print("ROC-AUC:", auc_aug)
print("PR-AUC:", pr_aug)
print("Accuracy:", acc_aug)



Baseline 2 - Real + Synthetic (pseudo-labels)
ROC-AUC: 0.6000000000000001
PR-AUC: 0.6595238095238095
Accuracy: 0.625
